<a href="https://colab.research.google.com/github/rishikksh20/FastSpeech2/blob/master/demo_fastspeech2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!git clone https://github.com/rishikksh20/FastSpeech2.git

Cloning into 'FastSpeech2'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (485/485), done.
remote: Compressing objects: 100% (338/338), done.
remote: Total 485 (delta 298), reused 314 (delta 138), pack-reused 0
Receiving objects: 100% (485/485), 3.83 MiB | 6.43 MiB/s, done.
Resolving deltas: 100% (298/298), done.


In [25]:
%cd FastSpeech2/

/content/FastSpeech2/FastSpeech2


In [26]:
# Use your drive and checkpoint here
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/fastspeech2/checkpoints/checkpoint_model_58k_steps.pyt'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
# Dependencies
!pip install unidecode
!pip install pyworld
!pip install configargparse

In [28]:
!nvcc --version # Check cuda version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
# Use appropriate cuda version for pytorch
!pip install --pre torch==1.6.0.dev20200623 -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html --upgrade

Looking in links: https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html
     |███████████████████████████████▋| 834.1MB 1.2MB/s eta 0:00:08tcmalloc: large alloc 1147494400 bytes == 0x65724000 @  0x7f4e7ca92615 0x591d67 0x4cc329 0x4cc48b 0x50a35c 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x58e5a3 0x50c227 0x58e5a3 0x50c227 0x58e5a3 0x50c227 0x58e5a3 0x5f76a2 0x4e5463 0x5513d1 0x5a9cbc 0x50a5c3 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x507d64
     |████████████████████████████████| 843.1MB 22kB/s 
ERROR: torchvision 0.8.0.dev20200701+cpu has requirement torch==1.7.0.dev20200701+cpu, but you'll have torch 1.6.0.dev20200623+cu101 which is incompatible.
  Found existing installation: torch 1.6.0.dev20200623
    Uninstalling torch-1.6.0.dev20200623:
      Successfully uninstalled torch-1.6.0.dev20200623


In [29]:
from fastspeech import FeedForwardTransformer
import hparams as hp
from dataset.texts import  phonemes_to_sequence
import numpy as np
import time
import configargparse
import logging
import os
import torch
import sys
import matplotlib.pyplot as plt
import IPython
import IPython.display as ipd

In [30]:
torch.cuda.is_available()

True

In [31]:
def synthesis(text, path):
    """Decode with E2E-TTS model."""
    
    print("TTS synthesis")
    # read training config
    idim = hp.symbol_len
    odim = hp.num_mels
    model = FeedForwardTransformer(idim, odim)

    if os.path.exists(path):
        logging.info('\nSynthesis Session...\n')
        model.load_state_dict(torch.load(path), strict=False)
    else:
        logging.info("Checkpoint not exixts")
        return None

    model.eval()

    # set torch device
    device = torch.device("cuda")
    model = model.to(device)
    print("Text :",text)
    input = np.asarray(phonemes_to_sequence(text.split()))
    print("Input :",input)
    text = torch.LongTensor(input)
    text = text.cuda()
    #[num_char]
    with torch.no_grad():
        # decode and write
        idx = input[:5]
        start_time = time.time()
        print("pridicting")
        outs, probs, att_ws = model.inference(text, None)

        logging.info("inference speed = %s msec / frame." % (
            (time.time() - start_time) / (int(outs.size(0)) * 1000)))
        if outs.size(0) == text.size(0) * 5:
            logging.warning("output length reaches maximum length .")

        mel = outs#.cpu().numpy() # [T_out, num_mel]

        return mel

In [32]:
# Only phoneme supports now, you can use any library to convert text to phonemes (CMUDict)
# Text : "a trade almost openly countenanced when "subjects" for the anatomy schools were only to be got by rifling graves, or worse.""
text = "AH0 T R EY1 D pau AO1 L M OW1 S T OW1 P AH0 N L IY0 K AW1 N T AH0 N AH0 N S T W EH1 N pau S AH1 B JH IH0 K T S pau F EU0 R DH IY0 AH0 N AE1 T AH0 M IY0 S K UW1 L Z W EU1 R OW1 N L IY0 T IH0 B IY0 G AA1 T B AY1 R AY1 F L IH0 NG G R EY1 V Z pau AO1 R W EU1 R S pau"

In [33]:
audio = synthesis(text, path)
m = audio.T

TTS synthesis
Text : AH0 T R EY1 D pau AO1 L M OW1 S T OW1 P AH0 N L IY0 K AW1 N T AH0 N AH0 N S T W EH1 N pau S AH1 B JH IH0 K T S pau F EU0 R DH IY0 AH0 N AE1 T AH0 M IY0 S K UW1 L Z W EU1 R OW1 N L IY0 T IH0 B IY0 G AA1 T B AY1 R AY1 F L IH0 NG G R EY1 V Z pau AO1 R W EU1 R S pau
Input : [ 5 44 41 22 15 55  8 32 33 37 42 44 37 40  5 34 32 28 31 10 34 44  5 34
  5 34 42 44 51 18 34 55 42  6 13 30 26 31 44 42 55 23 19 41 16 28  5 34
  4 44  5 33 28 42 31 49 32 53 51 20 41 37 34 32 28 44 26 13 28 24  2 44
 13 12 41 12 23 32 26 35 24 41 22 50 53 55  8 41 51 20 41 42 55]
pridicting


In [34]:
m.shape

torch.Size([80, 720])

In [35]:
fig = plt.figure(figsize=(12, 6))
plt.imshow(np.flip(m.cpu().numpy(), axis=0), interpolation='nearest', aspect='auto')

plt.show()

In [36]:
mel =  m.unsqueeze(0)
mel.shape

torch.Size([1, 80, 720])

In [37]:
vocoder = torch.hub.load('seungwonpark/melgan', 'melgan')

Using cache found in /root/.cache/torch/hub/seungwonpark_melgan_master


In [38]:
vocoder.eval()
if torch.cuda.is_available():
    vocoder = vocoder.cuda()
    mel = mel.cuda()

with torch.no_grad():
    audio = vocoder.inference(mel)

In [39]:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(audio.cpu().float().numpy(), sr=22050)

In [40]:
ipd.Audio(audio.cpu().float().numpy(), rate=22050) # load a NumPy array